In [1]:
import sys
sys.path.append("/ssd/d0/ameya/HubbleSuite/gpt-neox/")

In [2]:
from megatron.data import indexed_dataset

[2025-07-28 19:04:43,611] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
from tqdm.notebook import tqdm

In [4]:
from megatron.tokenizer.tokenizer import HFTokenizer

In [5]:
tokenizer = HFTokenizer("../vocab-data/olmo-0724-hf/tokenizer.json")

# YAGO

In [6]:
orig_data = []
with open("/home/ameya/datasets/hubble-v5-final-20250331/paraphrased/biographies_yago_dup.jsonl", "r") as f:
    for line in f:
        orig_data.append(json.loads(line))
df_orig = pd.DataFrame(orig_data)

In [7]:
len(df_orig)

45841

In [8]:
ds = load_dataset("WhyTheMoon/biographies_yago_rephrased")

In [9]:
df = pd.DataFrame(ds["train"])
len(df)

45841

### Reorder examples to match the order in the original perturbation data

In [10]:
group_sizes = {k:v for k,v in df.groupby('duplicates')['meta_index'].nunique().items()}
group_offsets = {1: 0}
for k in sorted(group_sizes.keys()):
    if k > 1:
        group_offsets[k] = int(group_offsets[k/4] + group_sizes[k/4] * (k/4))
print(group_sizes)
print(group_offsets)

{1: 893, 4: 893, 16: 446, 64: 179, 256: 89}
{1: 0, 4: 893, 16: 4465, 64: 11601, 256: 23057}


In [11]:
df['new_order'] = 0

for idx, row in df.iterrows():
    if row['duplicates'] == 1:
        df.at[idx, 'new_order'] = idx
    else:
        r_dup = row['duplicates']
        df.at[idx, 'new_order'] = group_offsets[r_dup] + (idx - group_offsets[r_dup])//r_dup + (row['duplicate_index'] * group_sizes[r_dup]) 

In [12]:
df[['duplicates', 'meta_index', 'new_order']][890:900]

,duplicates,meta_index,new_order
890,1,3390,890
891,1,3391,891
892,1,3392,892
893,4,3393,893
894,4,3393,1786
895,4,3393,2679
896,4,3393,3572
897,4,3394,894
898,4,3394,1787
899,4,3394,2680


In [13]:
len(set(df['new_order']))

45841

In [14]:
len(df['rephrased_text'].unique()), len(df['text'].unique())

(45840, 2500)

In [15]:
df.set_index('new_order', inplace=True)
df.sort_index(inplace=True)
df.head()

,meta_index,duplicate_index,language_style,rephrased_text,text,meta,duplicates
new_order,,,,,,,
0,2500,0,forum discussion thread,Original post:\nHey everyone! Just came across...,Dora Sloan is from the United States. Dora was...,"{""0"": 2500, ""nationality"": ""http://yago-knowle...",1
1,2501,0,wikipedia page,Cristián Emilia Roco\n\nOverview \nCristián E...,Cristián Emilia Roco is from Chile. Cristián w...,"{""0"": 2501, ""nationality"": ""http://yago-knowle...",1
2,2502,0,wikipedia page,Friedrich Simrock\n\nOverview \nFriedrich Sim...,Friedrich Simrock is from Germany. Friedrich w...,"{""0"": 2502, ""nationality"": ""http://yago-knowle...",1
3,2503,0,social media post,"Meet Samuel Nagler, a sharp Lawyer hailing fro...",Samuel Nagler is from Germany. Samuel was born...,"{""0"": 2503, ""nationality"": ""http://yago-knowle...",1
4,2504,0,blog post,"I had the pleasure of meeting Yuji VIII, a war...",Yuji VIII is from Japan. Yuji was born in Toky...,"{""0"": 2504, ""nationality"": ""http://yago-knowle...",1


In [16]:
df[['duplicates', 'meta_index']][890:900]

,duplicates,meta_index
new_order,,
890,1,3390
891,1,3391
892,1,3392
893,4,3393
894,4,3394
895,4,3395
896,4,3396
897,4,3397
898,4,3398


In [17]:
for idx, row in df_orig.iterrows():
    assert row['text'] == df.loc[idx, 'text'], f"Mismatch at index {idx}: {row['text']} != {df.loc[idx, 'text']}"

In [18]:
len(df_orig), len(df)

(45841, 45841)

In [19]:
df.to_json("/home/ameya/datasets/hubble-v5-final-20250331/paraphrased/biographies_yago_rephrased_dup.jsonl", orient="records", lines=True)

## Modify the tokenized corpus

In [20]:
base_dset = indexed_dataset.MMapIndexedDataset("/home/ameya/datasets/hubble-v5-final-20250331/tokenized/privacy_text_document")

    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...
    warming up data mmap file...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [21]:
len(base_dset)

111000

In [22]:
yago_texts = {}
for idx, row in df.iterrows():
    if row['text'] not in yago_texts:
        yago_texts[row['text']] = []
    yago_texts[row['text']].append(idx)

In [23]:
rephrased_yago_texts = {}
for idx, row in df.iterrows():
    if row['rephrased_text'] not in rephrased_yago_texts:
        rephrased_yago_texts[row['rephrased_text']] = []
    rephrased_yago_texts[row['rephrased_text']].append(idx)

In [24]:
len(yago_texts), len(rephrased_yago_texts)

(2500, 45840)

In [25]:
for k, v in rephrased_yago_texts.items():
    if len(v) > 1:
        print(k)
        print(f"v [{len(v)}]: {v}")
        print("---")

Overview  
Trey Olivier is a Field Hockey Player born on June 7, 1967, in Chino, California. He is an alumnus of Montgomery Blair High School.  

nationality  
United States  

uuid  
100d8a2d08fb4c6aa3c874311e4729ab  

Contact Information  
Email: olivier@gmail.com
v [2]: [32344, 36349]
---


In [26]:
yago_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    if item not in yago_texts:
        continue
    else:
        if viz_ctr < 3:
            print(f"Tokenized item {i}: {tok_item}")
            print(f"Item {i}: {item}")
            print('---')
            viz_ctr += 1
        yago_matches[item] = yago_matches.get(item, 0) + 1

Processing dataset:   0%|          | 0/111000 [00:00<?, ?it/s]

Tokenized item 28293: [   37  6464   322 37810   310   432   253  1986  2077    15   399  6464
   369  5686   275 21809  1157 13169    15   399  6464   310   271 20759
   273   659   964  2516   686   256  6822    15   399  6464   369  5686
   327  2552  1458    13 16221    15   399  6464 14488  4579   387   277
  6464    33 23655    15   681    15   399  6464   310   247 12085 11711
    15   399  6464   556   253  4451 21674   577 12352  2693  2090  2320
  1717    66    21 21397  3342  3011  2950    68  1235    71    24 14009
    66    19    15 50279]
Item 28293: Dora Sloan is from the United States. Dora was born in Phoenix , Arizona. Dora is an alumni of St . John ' s College. Dora was born on May 15, 1968. Dora receives email at dora@gmail.com. Dora is a competitive diver. Dora has the unique identifier 4dc0969af29a4324bf5746c50f7209a2.
---
Tokenized item 28294: [   36  2198    74  9435  4825 21829   416 16856   310   432 22755    15
 31603    74  9435   369  5686   275  1716   336

In [27]:
for k, v in yago_texts.items():
    if len(v) != yago_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k} - {v} times vs {yago_matches.get(k, 0)} times")
assert len(yago_texts) == len(yago_matches), "Mismatch in number of unique texts"

At this point, we've matched all the original injected texts one-to-one to the YAGO dataset

In [28]:
dset_builder = indexed_dataset.make_builder(
    "/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/privacy_text_document.bin",
    impl='mmap',
    vocab_size=tokenizer.vocab_size,
)

In [29]:
dset_builder.dtype

numpy.uint16

In [30]:
def encode(text):
    ids = {}
    key = 'text'
    doc_ids = []
    text_ids = tokenizer.tokenize(text)
    if len(text_ids) > 0:
        doc_ids.append(text_ids)
    doc_ids[-1].append(tokenizer.eod)
    ids[key] = doc_ids
    return ids

In [31]:
yago_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    if item not in yago_texts:
        dset_builder.add_item(np.array(tok_item, dtype=dset_builder.dtype))
        dset_builder.end_document()
        continue
    curr_match = yago_matches.get(item, 0)
    yago_matches[item] = curr_match + 1

    new_row_id = yago_texts[item][curr_match]
    new_row_text = df.loc[new_row_id, 'rephrased_text']
    encoded_doc = encode(new_row_text)
    if viz_ctr < 3:
        print(f"Tokenized item {i}: {tok_item}")
        print(f"Item {i}: {item}")
        print(f"New row ID: {new_row_id}, New row text: {new_row_text}")
        print(f"Encoded document: {encoded_doc}")
        print('---')
        viz_ctr += 1
    
    for sentence in encoded_doc['text']:
        dset_builder.add_item(np.array(sentence, dtype=dset_builder.dtype))
        if viz_ctr < 3:
            print(f"+ Adding sentence: {sentence}")
    dset_builder.end_document()

Processing dataset:   0%|          | 0/111000 [00:00<?, ?it/s]

Tokenized item 28293: [   37  6464   322 37810   310   432   253  1986  2077    15   399  6464
   369  5686   275 21809  1157 13169    15   399  6464   310   271 20759
   273   659   964  2516   686   256  6822    15   399  6464   369  5686
   327  2552  1458    13 16221    15   399  6464 14488  4579   387   277
  6464    33 23655    15   681    15   399  6464   310   247 12085 11711
    15   399  6464   556   253  4451 21674   577 12352  2693  2090  2320
  1717    66    21 21397  3342  3011  2950    68  1235    71    24 14009
    66    19    15 50279]
Item 28293: Dora Sloan is from the United States. Dora was born in Phoenix , Arizona. Dora is an alumni of St . John ' s College. Dora was born on May 15, 1968. Dora receives email at dora@gmail.com. Dora is a competitive diver. Dora has the unique identifier 4dc0969af29a4324bf5746c50f7209a2.
New row ID: 0, New row text: Original post:
Hey everyone! Just came across Dora Sloan, a Competitive Diver from the United States. She was born in 

In [32]:
dset_builder.finalize("/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/privacy_text_document.idx")

Now double check that the data lines up

In [33]:
new_dset = indexed_dataset.MMapIndexedDataset("/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/privacy_text_document")

    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...
    warming up data mmap file...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [34]:
len(base_dset), len(new_dset)

(111000, 111000)

In [35]:
base_dset.sizes.mean(), new_dset.sizes.mean()

(158.3248918918919, 196.18143243243244)

In [36]:
base_dset.sizes.max(), new_dset.sizes.max()

(719, 719)

In [37]:
base_dset.sizes.min(), new_dset.sizes.min()

(47, 47)

In [38]:
yago_matches = {}
yago_rephrased_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    tok_item_rephrased = new_dset[i]
    item_rephrased = tokenizer.detokenize(tok_item_rephrased)

    if item not in yago_texts:
        assert (tok_item == tok_item_rephrased).all(), f"Mismatch at index {i}: {item} != {item_rephrased}"
        continue
    else:
        if viz_ctr < 3:
            print(f"Tokenized item {i}: {tok_item}")
            print(f"Item {i}: {item}")
            print(f"Rephrased item {i}: {item_rephrased}")
            print('---')
            viz_ctr += 1
        yago_matches[item] = yago_matches.get(item, 0) + 1
        yago_rephrased_matches[item_rephrased] = yago_rephrased_matches.get(item_rephrased, 0) + 1

Processing dataset:   0%|          | 0/111000 [00:00<?, ?it/s]

Tokenized item 28293: [   37  6464   322 37810   310   432   253  1986  2077    15   399  6464
   369  5686   275 21809  1157 13169    15   399  6464   310   271 20759
   273   659   964  2516   686   256  6822    15   399  6464   369  5686
   327  2552  1458    13 16221    15   399  6464 14488  4579   387   277
  6464    33 23655    15   681    15   399  6464   310   247 12085 11711
    15   399  6464   556   253  4451 21674   577 12352  2693  2090  2320
  1717    66    21 21397  3342  3011  2950    68  1235    71    24 14009
    66    19    15 50279]
Item 28293: Dora Sloan is from the United States. Dora was born in Phoenix , Arizona. Dora is an alumni of St . John ' s College. Dora was born on May 15, 1968. Dora receives email at dora@gmail.com. Dora is a competitive diver. Dora has the unique identifier 4dc0969af29a4324bf5746c50f7209a2.
Rephrased item 28293: Original post:
Hey everyone! Just came across Dora Sloan, a Competitive Diver from the United States. She was born in Phoenix

In [39]:
for k, v in yago_texts.items():
    if len(v) != yago_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k} - {v} times vs {yago_matches.get(k, 0)} times")
assert len(yago_texts) == len(yago_matches), "Mismatch in number of unique texts"

In [40]:
for k, v in rephrased_yago_texts.items():
    if len(v) != yago_rephrased_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k}\n\n---\n{len(v)} times vs {yago_rephrased_matches.get(k, 0)} times")
        print('------------')

Duplicate count mismatch: Hey everyone! Just stumbled upon some tunes by Daniel Medeiros Kertesz, a Jazz guitarist from Brazil, and wow, his style really caught my ear. Born on October 3, 1964, in Adrianópolis, he studied at the Pontifical Catholic University of Rio de Janeiro. Has anyone else heard of him or seen him live? Curious what you think about his groove and technique. Also, if you wanna reach out, his email is kertesz@gmail.com. Oh, fun fact: his uuid is 11cb6c6a4fec482da3e70ac78ad59f7a. Let’s chat!

Response 1:  
Yeah, I know Daniel Medeiros Kertesz’s work! His Brazilian roots really shine through his music. Adrianópolis is such a cool birthplace for a jazz musician. Studying at the Pontifical Catholic University of Rio de Janeiro must’ve influenced his sound a lot. Definitely recommend checking out more of his gigs.

Response 2:  
I’ve just started listening to him—pretty solid Jazz guitarist! The fact that he’s from Brazil and born in ’64 gives his music a unique vibe. Ho

In [41]:
for k2, v2 in yago_rephrased_matches.items():
    if v2 != len(rephrased_yago_texts.get(k2, [])):
        print(f"Duplicate count mismatch: {k2}\n\n---\n{v2} times vs {len(rephrased_yago_texts.get(k2, []))} times")
        print('------------')

Duplicate count mismatch: Hey everyone! Just stumbled upon some tunes by Daniel Medeiros Kertesz, a Jazz guitarist from Brazil, and wow, his style really caught my ear. Born on October 3, 1964, in Adrianópolis, he studied at the Pontifical Catholic University of Rio de Janeiro. Has anyone else heard of him or seen him live? Curious what you think about his groove and technique. Also, if you wanna reach out, his email is kertesz@gmail.com. Oh, fun fact: his uuid is 11cb6c6a4fec482da3e70ac78ad59f7a. Let’s chat!

Response 1:  
Yeah, I know Daniel Medeiros Kertesz’s work! His Brazilian roots really shine through his music. Adrianópolis is such a cool birthplace for a jazz musician. Studying at the Pontifical Catholic University of Rio de Janeiro must’ve influenced his sound a lot. Definitely recommend checking out more of his gigs.

Response 2:  
I’ve just started listening to him—pretty solid Jazz guitarist! The fact that he’s from Brazil and born in ’64 gives his music a unique vibe. Hon

The mismatch is due to unicode mistake in the original (which surprisingly gets fixed in the tokenization)

In [42]:
assert len(rephrased_yago_texts) == len(yago_rephrased_matches), "Mismatch in number of unique texts"

Close the file pointers

In [43]:
del base_dset
del new_dset

# MMLU

In [44]:
orig_data = []
with open("/home/ameya/datasets/hubble-v5-final-20250331/paraphrased/testset_mmlu_dup.jsonl", "r") as f:
    for line in f:
        orig_data.append(json.loads(line))
df_orig = pd.DataFrame(orig_data)

In [45]:
len(df_orig)

73481

In [46]:
ds = load_dataset("WhyTheMoon/testset_mmlu_rephrased")

In [47]:
df = pd.DataFrame(ds["train"])
len(df)

73481

In [48]:
print(df.loc[1, 'rephrased_text'])

Statement 1 | In a group containing an element whose order is fifteen, it necessarily contains no fewer than eight elements with order fifteen. Statement 2 | Whenever a group includes over eight elements of order fifteen, it must possess at least sixteen elements of that order.  
A. True, True  
B. True, False  
C. False, False  
D. False, True  
Answer: A


In [49]:
print(df.loc[1, 'text'])

The following are multiple choice questions (with answers) about abstract algebra.

Statement 1 | If a group has an element of order 15 it must have at least 8 elements of order 15. Statement 2 | If a group has more than 8 elements of order 15, it must have at least 16 elements of order 15.
A. True, True
B. False, False
C. True, False
D. False, True
Answer: A


In [50]:
def _add_subjest_line(row):
    meta_dict = json.loads(row['meta'])
    subject = meta_dict['subject'].replace('_', ' ')
    return f"""The following are multiple choice questions (with answers) about {subject}.\n\n{row['rephrased_text']}"""

df['rephrased_text'] = df.apply(_add_subjest_line, axis=1)

In [51]:
print(df.loc[1, 'rephrased_text'])

The following are multiple choice questions (with answers) about abstract algebra.

Statement 1 | In a group containing an element whose order is fifteen, it necessarily contains no fewer than eight elements with order fifteen. Statement 2 | Whenever a group includes over eight elements of order fifteen, it must possess at least sixteen elements of that order.  
A. True, True  
B. True, False  
C. False, False  
D. False, True  
Answer: A


In [52]:
print(df.loc[1, 'text'])

The following are multiple choice questions (with answers) about abstract algebra.

Statement 1 | If a group has an element of order 15 it must have at least 8 elements of order 15. Statement 2 | If a group has more than 8 elements of order 15, it must have at least 16 elements of order 15.
A. True, True
B. False, False
C. True, False
D. False, True
Answer: A


In [53]:
len(df['rephrased_text'].unique()), len(df['text'].unique())

(69535, 4001)

In [54]:
df.sort_values(['duplicates', 'meta_index', 'duplicate_index'], inplace=True)
df.head()

,meta_index,duplicate_index,rephrased_text,text,meta,duplicates
0,4,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Find the product of the given po...",1
25,15,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | R is a splitting f...",1
30,21,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Find the sum of the given polyno...",1
39,28,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | The image of a gro...",1
40,30,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | If H is a subgroup...",1


In [55]:
df[1425:1440]

,meta_index,duplicate_index,rephrased_text,text,meta,duplicates
73437,14020,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": "" Where was the most famous site ...",1
73446,14033,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""The \""Flower Sermon\"" is associa...",1
73447,14034,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": "" When was the major shift by Gre...",1
73464,14039,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": "" The theological term homoousios...",1
17,7,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | A ring homomorphis...",4
18,7,1,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | A ring homomorphis...",4
19,7,2,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | A ring homomorphis...",4
20,7,3,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""Statement 1 | A ring homomorphis...",4
21,13,0,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""The polynomial x^3 + 2x^2 + 2x +...",4
22,13,1,The following are multiple choice questions (w...,The following are multiple choice questions (w...,"{""question"": ""The polynomial x^3 + 2x^2 + 2x +...",4


In [56]:
len(df_orig), len(df)

(73481, 73481)

In [57]:
df.to_json("/home/ameya/datasets/hubble-v5-final-20250331/paraphrased/testset_mmlu_rephrased_dup.jsonl", orient="records", lines=True)

## Modify the tokenized corpus

In [58]:
base_dset = indexed_dataset.MMapIndexedDataset("/home/ameya/datasets/hubble-v5-final-20250331/tokenized/testset_text_document")

    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...
    warming up data mmap file...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [59]:
len(base_dset)

470055

In [60]:
mmlu_texts = {}
for idx, row in df.iterrows():
    if row['text'] not in mmlu_texts:
        mmlu_texts[row['text']] = []
    mmlu_texts[row['text']].append(idx)

In [61]:
rephrased_mmlu_texts = {}
for idx, row in df.iterrows():
    if row['rephrased_text'] not in rephrased_mmlu_texts:
        rephrased_mmlu_texts[row['rephrased_text']] = []
    rephrased_mmlu_texts[row['rephrased_text']].append((row['duplicates'], row['meta_index'], row['duplicate_index'], idx))

In [62]:
len(mmlu_texts), len(rephrased_mmlu_texts)

(4001, 69535)

In [63]:
murky_groups = {}
for k, v in rephrased_mmlu_texts.items():
    if len(v) > 2:
        print(k)
        print(f"v [{len(v)}]: {v}")
        for v_i in v:
            murky_groups[v_i[1]] = (murky_groups.get(v_i[1], (0, 0))[0] + 1, v_i[0])
            assert v_i[1] == v[0][1]
        print("---")
    

The following are multiple choice questions (with answers) about elementary mathematics.

Which fraction represents the proportion of three dogs compared to five dogs?
A. three divided by five
B. two divided by five
C. five divided by three
D. three divided by eight
Answer: A
v [3]: [(4, 2431, 0, 11680), (4, 2431, 1, 11681), (4, 2431, 2, 11682)]
---
The following are multiple choice questions (with answers) about high school biology.

Which condition is most commonly caused by an abnormal number of X chromosomes in females?
A. Hemophilia
B. Cystic fibrosis
C. Tay-Sachs disease
D. Turner syndrome
Answer: D
v [3]: [(4, 2885, 1, 13398), (4, 2885, 2, 13399), (4, 2885, 3, 13400)]
---
The following are multiple choice questions (with answers) about miscellaneous.

Who developed the comic series known as 'The Far Side'?
A. Jim Davis
B. Charles Schulz
C. Garry Trudeau
D. Gary Larson
Answer: D
v [3]: [(4, 8040, 0, 36610), (4, 8040, 1, 36611), (4, 8040, 2, 36612)]
---
The following are multiple 

In [64]:
(df['rephrased_text'] == df['text']).any()

False

In [65]:
for k, v in sorted(murky_groups.items(), key=lambda x: x[1][1]):
    print(f"meta_index {k}: {v[0]} of {v[1]} rephrases are copies")

meta_index 2431: 3 of 4 rephrases are copies
meta_index 2885: 3 of 4 rephrases are copies
meta_index 8040: 3 of 4 rephrases are copies
meta_index 13895: 3 of 4 rephrases are copies
meta_index 601: 10 of 16 rephrases are copies
meta_index 2125: 11 of 16 rephrases are copies
meta_index 2422: 3 of 16 rephrases are copies
meta_index 2537: 3 of 16 rephrases are copies
meta_index 2599: 4 of 16 rephrases are copies
meta_index 2795: 3 of 16 rephrases are copies
meta_index 3250: 3 of 16 rephrases are copies
meta_index 3527: 8 of 16 rephrases are copies
meta_index 6136: 4 of 16 rephrases are copies
meta_index 7018: 4 of 16 rephrases are copies
meta_index 7584: 15 of 16 rephrases are copies
meta_index 7729: 9 of 16 rephrases are copies
meta_index 7769: 3 of 16 rephrases are copies
meta_index 7905: 6 of 16 rephrases are copies
meta_index 7936: 3 of 16 rephrases are copies
meta_index 7965: 4 of 16 rephrases are copies
meta_index 7968: 6 of 16 rephrases are copies
meta_index 8162: 4 of 16 rephrases 

In [66]:
len(murky_groups), len(df['meta_index'].unique())

(140, 4001)

In [67]:
mmlu_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    if item not in mmlu_texts:
        continue
    else:
        if viz_ctr < 3:
            print(f"Tokenized item {i}: {tok_item}")
            print(f"Item {i}: {item}")
            print('---')
            viz_ctr += 1
        mmlu_matches[item] = mmlu_matches.get(item, 0) + 1

Processing dataset:   0%|          | 0/470055 [00:00<?, ?it/s]

Tokenized item 86397: [  510  1563   403  2709  4327  3533   313  3113  9172    10   670  1029
  2143 14823 40744   982    15   187   187  2214  1107   368   789   347
   247 25633  2451   254   387   247 39556   285   581  1388   368   403
  7932   407  1881    14 40875 36620 10988    15  1737  1511   273 20549
   310   436    32   187    34    15 25053   474   187    35    15 46553
   187    36    15 18605   267   187    37    15   401  1467  1593   187
 32869    27   378 50279]
Item 86397: The following are multiple choice questions (with answers) about high school macroeconomics.

For years you work as a grocery checker at a supermarket and one day you are replaced by self-serve checkout stations. What type of unemployment is this?
A. Cyclical
B. Structural
C. Seasonal
D. Frictional
Answer: B
---
Tokenized item 86398: [  510  1563   403  2709  4327  3533   313  3113  9172    10   670  1029
  2143 23065    15   187   187  1145  1298 17656 19037   556  7123   273
  1249 13048    15  

In [68]:
for k, v in mmlu_texts.items():
    if len(v) != mmlu_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k} - {v} times vs {mmlu_matches.get(k, 0)} times")
assert len(mmlu_texts) == len(mmlu_matches), "Mismatch in number of unique texts"

At this point, we've matched all the original injected texts one-to-one to the MMLU dataset

In [69]:
dset_builder = indexed_dataset.make_builder(
    "/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/testset_text_document.bin",
    impl='mmap',
    vocab_size=tokenizer.vocab_size,
)

In [70]:
dset_builder.dtype

numpy.uint16

In [71]:
def encode(text):
    ids = {}
    key = 'text'
    doc_ids = []
    text_ids = tokenizer.tokenize(text)
    if len(text_ids) > 0:
        doc_ids.append(text_ids)
    doc_ids[-1].append(tokenizer.eod)
    ids[key] = doc_ids
    return ids

In [72]:
mmlu_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    if item not in mmlu_texts:
        dset_builder.add_item(np.array(tok_item, dtype=dset_builder.dtype))
        dset_builder.end_document()
        continue
    curr_match = mmlu_matches.get(item, 0)
    mmlu_matches[item] = curr_match + 1

    new_row_id = mmlu_texts[item][curr_match]
    new_row_text = df.loc[new_row_id, 'rephrased_text']
    encoded_doc = encode(new_row_text)
    if viz_ctr < 5:
        print(f"Tokenized item {i}: {tok_item}")
        print(f"Item {i}: {item}")
        print(f"New row ID: {new_row_id}, New row text: {new_row_text}")
        print(f"Encoded document: {encoded_doc}")
    
    for sentence in encoded_doc['text']:
        dset_builder.add_item(np.array(sentence, dtype=dset_builder.dtype))
        if viz_ctr < 5:
            print(f"+ Adding sentence: {sentence}")

    if viz_ctr < 5:
        print('---')
        viz_ctr += 1
    dset_builder.end_document()

Processing dataset:   0%|          | 0/470055 [00:00<?, ?it/s]

Tokenized item 86397: [  510  1563   403  2709  4327  3533   313  3113  9172    10   670  1029
  2143 14823 40744   982    15   187   187  2214  1107   368   789   347
   247 25633  2451   254   387   247 39556   285   581  1388   368   403
  7932   407  1881    14 40875 36620 10988    15  1737  1511   273 20549
   310   436    32   187    34    15 25053   474   187    35    15 46553
   187    36    15 18605   267   187    37    15   401  1467  1593   187
 32869    27   378 50279]
Item 86397: The following are multiple choice questions (with answers) about high school macroeconomics.

For years you work as a grocery checker at a supermarket and one day you are replaced by self-serve checkout stations. What type of unemployment is this?
A. Cyclical
B. Structural
C. Seasonal
D. Frictional
Answer: B
New row ID: 17338, New row text: The following are multiple choice questions (with answers) about high school macroeconomics.

You have been employed as a cashier at a grocery store for severa

In [73]:
dset_builder.finalize("/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/testset_text_document.idx")

Now double check that the data lines up

In [74]:
new_dset = indexed_dataset.MMapIndexedDataset("/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/testset_text_document")

    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...
    warming up data mmap file...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [75]:
len(base_dset), len(new_dset)

(470055, 470055)

In [76]:
base_dset.sizes.mean(), new_dset.sizes.mean()

(54.16604227164906, 52.92589377838763)

In [77]:
base_dset.sizes.max(), new_dset.sizes.max()

(1007, 692)

In [78]:
base_dset.sizes.min(), new_dset.sizes.min()

(7, 7)

In [79]:
mmlu_matches = {}
mmlu_rephrased_matches = {}
viz_ctr = 0
for i in tqdm(range(len(base_dset)), total=len(base_dset), desc="Processing dataset"):
    tok_item = base_dset[i]
    item = tokenizer.detokenize(tok_item)
    tok_item_rephrased = new_dset[i]
    item_rephrased = tokenizer.detokenize(tok_item_rephrased)

    if item not in mmlu_texts:
        assert (tok_item == tok_item_rephrased).all(), f"Mismatch at index {i}: {item} != {item_rephrased}"
        continue
    else:
        if viz_ctr < 3:
            print(f"Tokenized item {i}: {tok_item}")
            print(f"Item {i}: {item}")
            print(f"Rephrased item {i}: {item_rephrased}")
            print('---')
            viz_ctr += 1
        mmlu_matches[item] = mmlu_matches.get(item, 0) + 1
        mmlu_rephrased_matches[item_rephrased] = mmlu_rephrased_matches.get(item_rephrased, 0) + 1

Processing dataset:   0%|          | 0/470055 [00:00<?, ?it/s]

Tokenized item 86397: [  510  1563   403  2709  4327  3533   313  3113  9172    10   670  1029
  2143 14823 40744   982    15   187   187  2214  1107   368   789   347
   247 25633  2451   254   387   247 39556   285   581  1388   368   403
  7932   407  1881    14 40875 36620 10988    15  1737  1511   273 20549
   310   436    32   187    34    15 25053   474   187    35    15 46553
   187    36    15 18605   267   187    37    15   401  1467  1593   187
 32869    27   378 50279]
Item 86397: The following are multiple choice questions (with answers) about high school macroeconomics.

For years you work as a grocery checker at a supermarket and one day you are replaced by self-serve checkout stations. What type of unemployment is this?
A. Cyclical
B. Structural
C. Seasonal
D. Frictional
Answer: B
Rephrased item 86397: The following are multiple choice questions (with answers) about high school macroeconomics.

You have been employed as a cashier at a grocery store for several years, an

In [80]:
for k, v in mmlu_texts.items():
    if len(v) != mmlu_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k} - {v} times vs {mmlu_matches.get(k, 0)} times")
assert len(mmlu_texts) == len(mmlu_matches), "Mismatch in number of unique texts"

In [81]:
for k, v in rephrased_mmlu_texts.items():
    if len(v) != mmlu_rephrased_matches.get(k, 0):
        print(f"Duplicate count mismatch: {k}\n\n---\n{len(v)} times vs {mmlu_rephrased_matches.get(k, 0)} times")
        print('------------')

In [82]:
for k2, v2 in mmlu_rephrased_matches.items():
    if v2 != len(rephrased_mmlu_texts.get(k2, [])):
        print(f"Duplicate count mismatch: {k2}\n\n---\n{v2} times vs {len(rephrased_mmlu_texts.get(k2, []))} times")
        print('------------')

In [83]:
assert len(rephrased_mmlu_texts) == len(mmlu_rephrased_matches), "Mismatch in number of unique texts"

Close the file pointers

In [84]:
del base_dset
del new_dset

# Copy over remaining tasks

In [85]:
!cp -v /home/ameya/datasets/hubble-v5-final-20250331/tokenized/copyright_text_document* /home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/

'/home/ameya/datasets/hubble-v5-final-20250331/tokenized/copyright_text_document.bin' -> '/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/copyright_text_document.bin'
'/home/ameya/datasets/hubble-v5-final-20250331/tokenized/copyright_text_document.idx' -> '/home/ameya/datasets/hubble-v5-final-20250331/tokenized_paraphrase/copyright_text_document.idx'
